In [12]:
# Imports
import torch
from torch import nn
from src.utils.preprocess_utils import midi_to_note_indices, notes_to_midi
import numpy as np
import torch.nn.functional as F
from mido import MidiFile, MidiTrack, Message
from datetime import datetime

In [13]:
# Define model architecture
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, hidden=None):
        # x: [batch_size, sequence_length]
        x = x.long()
        embedded = self.embedding(x)  # [batch_size, sequence_length, embedding_size]
        lstm_out, hidden = self.lstm(embedded, hidden)  # [batch_size, sequence_length, hidden_size]
        output = self.fc(lstm_out)  # [batch_size, sequence_length, vocab_size]
        
        return output

In [14]:
# Load model
model_name = "2025-01-03_22-07-53_e64_h128_l2_sl50"
model = torch.load(f'weights/model_2/{model_name}')
model.eval()

C:\Users\janikbalint\AppData\Local\Temp\ipykernel_4588\3153802850.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f'weights/model_2/{model_name}')


LSTMModel(
  (embedding): Embedding(85, 64)
  (lstm): LSTM(64, 128, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=85, bias=True)
)

In [15]:
# Load input
input = midi_to_note_indices("data/input.mid", 24, 108)
sequence_length = 200  # Desired length of the generated sequence
temperature = 0.3
print(input)

[24, 26, 31, 36]


In [16]:
def generate_sequence(model, start_sequence, sequence_length, num_notes, temperature=1.0):
    model.eval()

    start_tensor = torch.tensor(start_sequence, dtype=torch.long).unsqueeze(0)  # [1, sequence_length, input_size]

    generated_sequence = list(start_sequence)

    for i in range(sequence_length):
        with torch.no_grad():
            output = model(start_tensor)  # [1, i, output_size]
            logits = output.squeeze(0)  # Remove the batch dimension: [i, output_size]

            # Temperature scaling
            scaled_logits = logits
            if temperature != 0:
                scaled_logits = logits / temperature

            # Apply softmax to get probabilities for the next note
            output_probs = F.softmax(scaled_logits, dim=-1)

            # Sample the next note based on the probabilities
            next_note = torch.multinomial(output_probs[-1], 1).item()

            # Append the predicted note to the sequence and this sequence's one-hot
            # encoded format will be the input for the next note's generation
            generated_sequence.append(next_note)
            generated_sequence_tensor = np.array(generated_sequence)
            start_tensor = torch.tensor(generated_sequence_tensor, dtype=torch.long).unsqueeze(0)  # Add batch dimension
    
    return generated_sequence

generated_sequence = generate_sequence(model.cpu(), input, sequence_length, num_notes=85, temperature=temperature)

print("Generated Sequence:")
print(generated_sequence)

Generated Sequence:
[24, 26, 31, 36, 52, 45, 40, 47, 51, 31, 52, 32, 36, 48, 35, 43, 40, 40, 24, 36, 45, 53, 26, 53, 32, 52, 36, 50, 19, 47, 50, 38, 39, 36, 45, 39, 41, 36, 50, 35, 32, 33, 50, 40, 51, 52, 35, 31, 40, 45, 37, 33, 36, 48, 41, 38, 36, 37, 36, 47, 35, 50, 40, 52, 45, 48, 36, 48, 41, 38, 41, 42, 43, 33, 39, 37, 36, 36, 36, 53, 43, 29, 43, 33, 38, 35, 33, 48, 51, 38, 31, 29, 27, 33, 45, 40, 33, 33, 52, 36, 48, 31, 52, 40, 28, 29, 53, 36, 29, 36, 48, 45, 38, 36, 31, 48, 45, 40, 49, 48, 48, 52, 51, 53, 43, 38, 36, 43, 38, 48, 53, 33, 53, 43, 39, 39, 34, 32, 36, 42, 44, 36, 43, 52, 40, 47, 36, 39, 48, 54, 41, 48, 39, 34, 45, 45, 38, 50, 50, 38, 38, 29, 45, 39, 47, 35, 36, 52, 41, 39, 36, 40, 40, 43, 60, 40, 26, 33, 38, 31, 48, 39, 43, 48, 53, 40, 26, 34, 48, 36, 43, 48, 41, 40, 48, 31, 43, 42, 33, 40, 38, 36, 36, 53]


In [17]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_path = f"generations/model_2/{current_datetime}_t{temperature:.1f}_w{sequence_length}_M_{model_name}.mid"

notes_to_midi(generated_sequence, file_path)

MIDI file saved to generations/model_2/2025-01-04_09-39-07_t0.3_w200_M_2025-01-03_22-07-53_e64_h128_l2_sl50.mid
